In [1]:
import json
import pickle
import os
from collections import defaultdict
from tqdm import tqdm

from os import listdir
from os.path import isfile, join

### PHEME for NAACL2021

In [2]:
! ls ../data/pheme/all-rnr-annotated-threads/charliehebdo-all-rnr-threads

non-rumours  rumours


In [96]:
# topics = [
#             'charliehebdo',
#             'ferguson',
#             'ottawashooting',
#             'putinmissing',
#             'ebola-essien',
#             'germanwings-crash',
#             'prince-toronto',
#             'sydneysiege'
#          ]

In [4]:
'''convertion code from PHEME creators. Label: True = rumour is true. False = rumour is false'''
def convert_annotations(annotation, string = True):
    if 'misinformation' in annotation.keys() and 'true'in annotation.keys():
        if int(annotation['misinformation'])==0 and int(annotation['true'])==0:
            if string:
                label = "unverified"
            else:
                label = 2
        elif int(annotation['misinformation'])==0 and int(annotation['true'])==1 :
            if string:
                label = "true"
            else:
                label = 1
        elif int(annotation['misinformation'])==1 and int(annotation['true'])==0 :
            if string:
                label = "false"
            else:
                label = 0
        elif int(annotation['misinformation'])==1 and int(annotation['true'])==1:
            print ("OMG! They both are 1!")
            print(annotation['misinformation'])
            print(annotation['true'])
            label = None
            
    elif 'misinformation' in annotation.keys() and 'true' not in annotation.keys():
        # all instances have misinfo label but don't have true label
        if int(annotation['misinformation'])==0:
            if string:
                label = "unverified"
            else:
                label = 2
        elif int(annotation['misinformation'])==1:
            if string:
                label = "false"
            else:
                label = 0
                
    elif 'true' in annotation.keys() and 'misinformation' not in annotation.keys():
        print ('Has true not misinformation')
        label = None
    else:
        print('No annotations')
        label = None
           
    return label

In [ ]:
'''
    thread_anno_by_events - for NAACL2021
'''

base_path = "../data/pheme/all-rnr-annotated-threads"
all_event_thread_info = {}

events = [d for d in os.listdir(base_path) if d!='.DS_Store']
for event in tqdm(events):
    threads = [d for d in os.listdir('{}/{}'.format(base_path, event))]
    print(event, len(threads))

    thread_info = {}
    rumour_threads = [d for d in os.listdir('{}/{}/rumours'.format(base_path, event)) if d != '.DS_Store']

    for r_id in rumour_threads:
        # process lvl2 rumour veracity annotation (true, false, unverified rumour)
        with open('{}/{}/rumours/{}/annotation.json'.format(base_path, event, r_id)) as json_file:
            # anno fields: links - evidences/information about the rumour. category - rumour type. 
            # misinfo & true lables are used to determine labels for each thread
            anno = json.load(json_file)
            veracity_label = convert_annotations(anno)
            evidence_arr = anno['links']
            rumour_category = anno['category'] # TODO check what this is later 

        # combine lvl1 rumour detection labels with lvl2 labels
        if veracity_label != 'unverified':
            thread_info[r_id] = {
                'thread_id': r_id,
                'rumour_label': 'rumour',
                'veracity_label': veracity_label,
                'rumour_category': rumour_category
            }

    
    event_name = event.split('-')[0]
    all_event_thread_info[event_name]=thread_info

with open('./thread_annotations_by_event.json', 'w') as outfile:
    json.dump(all_event_thread_info, outfile)

# checking for the count
for k in all_event_thread_info:
    print(k, len(all_event_thread_info[k]))

In [5]:
'''
    thread_anno_by_events_three_class - for EMNLP2021
'''

base_path = "../data/pheme/all-rnr-annotated-threads"
all_event_thread_info = {}

events = [d for d in os.listdir(base_path) if d!='.DS_Store']
for event in tqdm(events):
    threads = [d for d in os.listdir('{}/{}'.format(base_path, event))]
    print(event, len(threads))

    thread_info = {}
    rumour_threads = [d for d in os.listdir('{}/{}/rumours'.format(base_path, event)) if d != '.DS_Store']

    for r_id in rumour_threads:
        # process lvl2 rumour veracity annotation (true, false, unverified rumour)
        with open('{}/{}/rumours/{}/annotation.json'.format(base_path, event, r_id)) as json_file:
            # anno fields: links - evidences/information about the rumour. category - rumour type. 
            # misinfo & true lables are used to determine labels for each thread
            anno = json.load(json_file)
            veracity_label = convert_annotations(anno)
            evidence_arr = anno['links']
            rumour_category = anno['category'] # TODO check what this is later 

        # combine lvl1 rumour detection labels with lvl2 labels
#         if veracity_label != 'unverified':
        thread_info[r_id] = {
            'thread_id': r_id,
            'rumour_label': 'rumour',
            'veracity_label': veracity_label,
            'rumour_category': rumour_category
        }

    
    event_name = event.split('-')[0]
    all_event_thread_info[event_name]=thread_info

with open('./thread_annotations_by_event_3class.json', 'w') as outfile:
    json.dump(all_event_thread_info, outfile)

# checking for the count
for k in all_event_thread_info:
    print(k, len(all_event_thread_info[k]))

  0%|          | 0/9 [00:00<?, ?it/s]

sydneysiege-all-rnr-threads 3


 11%|█         | 1/9 [00:00<00:06,  1.20it/s]

ottawashooting-all-rnr-threads 3


 22%|██▏       | 2/9 [00:01<00:05,  1.28it/s]

...

gurlitt-all-rnr-threads 3
putinmissing-all-rnr-threads 3
charliehebdo-all-rnr-threads 3


 89%|████████▉ | 8/9 [00:02<00:00,  2.19it/s]

germanwings-crash-all-rnr-threads 3


100%|██████████| 9/9 [00:03<00:00,  2.74it/s]

sydneysiege 522
ottawashooting 470
ebola 14
prince 229
ferguson 284
gurlitt 61
putinmissing 126
charliehebdo 458
germanwings 238


In [106]:
[
    "gurlitt",
    "germanwings",
    "ebola",
    "prince",
    "ferguson",
    "ottawashooting",
    "sydneysiege",
    "charliehebdo",
    "putinmissing"
]

['gurlitt',
 'germanwings',
 'ebola',
 'prince',
 'ferguson',
 'ottawashooting',
 'sydneysiege',
 'charliehebdo',
 'putinmissing']

In [133]:
with open('../data/preprocessed/id2text.pickle', 'rb') as handler:
    id2data = pickle.load(handler)

veracity_path = '../data/preprocessed/thread_annotations_by_event.json'
with open(veracity_path) as json_file:
    data = json.load(json_file)

# split the train/dev/test based on the event

# prepare test idx
test_event_name = 'gurlitt'
test = list(data[test_event_name].values())

# prepare train/dev idx
train_dev_list = []
for event_name in self.events:
    if event_name != self.test_event_name:
        train_dev_list.extend(list(data[event_name].values()))

self.train, self.dev = train_test_split(train_dev_list, test_size=0.15, random_state=0, shuffle=True)


#### Lvl 1 rumour verification anno ( rumour vs non-rumour) AND lvl2 rumour veracity anno (true, false, unverified) from PHEME data

In [99]:
'''convertion code from PHEME creators. Label: True = rumour is true. False = rumour is false'''
def convert_annotations(annotation, string = True):
    if 'misinformation' in annotation.keys() and 'true'in annotation.keys():
        if int(annotation['misinformation'])==0 and int(annotation['true'])==0:
            if string:
                label = "unverified"
            else:
                label = 2
        elif int(annotation['misinformation'])==0 and int(annotation['true'])==1 :
            if string:
                label = "true"
            else:
                label = 1
        elif int(annotation['misinformation'])==1 and int(annotation['true'])==0 :
            if string:
                label = "false"
            else:
                label = 0
        elif int(annotation['misinformation'])==1 and int(annotation['true'])==1:
            print ("OMG! They both are 1!")
            print(annotation['misinformation'])
            print(annotation['true'])
            label = None
            
    elif 'misinformation' in annotation.keys() and 'true' not in annotation.keys():
        # all instances have misinfo label but don't have true label
        if int(annotation['misinformation'])==0:
            if string:
                label = "unverified"
            else:
                label = 2
        elif int(annotation['misinformation'])==1:
            if string:
                label = "false"
            else:
                label = 0
                
    elif 'true' in annotation.keys() and 'misinformation' not in annotation.keys():
        print ('Has true not misinformation')
        label = None
    else:
        print('No annotations')
        label = None
           
    return label

In [ ]:
'thread anno w/o event'
thread_info = {}
events = [d for d in os.listdir(base_path) if d!='.DS_Store']
for event in tqdm(events):
    threads = [d for d in os.listdir('{}/{}'.format(base_path, event))]
    print(event, len(threads))

    rumour_threads = [d for d in os.listdir('{}/{}/rumours'.format(base_path, event)) if d != '.DS_Store']
    non_rumour_threads = [d for d in os.listdir('{}/{}/non-rumours'.format(base_path, event))]

    for r_id in rumour_threads:
        # process lvl2 rumour veracity annotation (true, false, unverified rumour)
        with open('{}/{}/rumours/{}/annotation.json'.format(base_path, event, r_id)) as json_file:
            # anno fields: links - evidences/information about the rumour. category - rumour type. 
            # misinfo & true lables are used to determine labels for each thread
            anno = json.load(json_file)
            veracity_label = convert_annotations(anno)
            evidence_arr = anno['links']
            rumour_category = anno['category'] # TODO check what this is later 

        # combine lvl1 rumour detection labels with lvl2 labels
        thread_info[r_id] = {
            'thread_id': r_id,
            'rumour_label': 'rumour',
            'veracity_label': veracity_label,
            'evidence': evidence_arr,
            'rumour_category': rumour_category
        }

    for r_id in non_rumour_threads:
        # non-rumour doesn't have lvl2 annotation.
        thread_info[r_id] = {'thread_id': r_id, 'rumour_label':'non-rumour'}
    
    event_name = event.split('-')[0]
    
# with open('./thread_annotations.json', 'w') as outfile:
#     json.dump(thread_info, outfile)

# # checking for the count
# for k in thread_info:
#     print(k, len(thread_info[k]))

In [ ]:
'thread_anno_by_events'
all_event_thread_info = {}

events = [d for d in os.listdir(base_path) if d!='.DS_Store']
for event in tqdm(events):
    threads = [d for d in os.listdir('{}/{}'.format(base_path, event))]
    print(event, len(threads))

    thread_info = {}
    rumour_threads = [d for d in os.listdir('{}/{}/rumours'.format(base_path, event)) if d != '.DS_Store']
    non_rumour_threads = [d for d in os.listdir('{}/{}/non-rumours'.format(base_path, event))]

    for r_id in rumour_threads:
        # process lvl2 rumour veracity annotation (true, false, unverified rumour)
        with open('{}/{}/rumours/{}/annotation.json'.format(base_path, event, r_id)) as json_file:
            # anno fields: links - evidences/information about the rumour. category - rumour type. 
            # misinfo & true lables are used to determine labels for each thread
            anno = json.load(json_file)
            veracity_label = convert_annotations(anno)
            evidence_arr = anno['links']
            rumour_category = anno['category'] # TODO check what this is later 

        # combine lvl1 rumour detection labels with lvl2 labels
        thread_info[r_id] = {
            'thread_id': r_id,
            'rumour_label': 'rumour',
            'veracity_label': veracity_label,
            'evidence': evidence_arr,
            'rumour_category': rumour_category
        }

    for r_id in non_rumour_threads:
        # non-rumour doesn't have lvl2 annotation.
        thread_info[r_id] = {'thread_id': r_id, 'rumour_label':'non-rumour'}
    
    event_name = event.split('-')[0]
    all_event_thread_info[event_name]=thread_info
    
# with open('./thread_annotations_by_event.json', 'w') as outfile:
#     json.dump(all_event_thread_info, outfile)

# # checking for the count
# for k in all_event_thread_info:
#     print(k, len(all_event_thread_info[k]))

#### Lvl 3 tweet level annotation for Stance detection (from PHEME-journalism data)

In [3]:
tweet_level_annotation = root_path + '/pheme/annotations/en-scheme-annotations.json'

tweet2stance = defaultdict(dict)
for _, line in enumerate(open(tweet_level_annotation, 'r')):
    line = json.loads(line)
    
    threadid = line['threadid']
    tweetid = line['tweetid']
    
    if threadid == tweetid: # source tweet
        tweet2stance[threadid][tweetid] = {'thread_id': threadid, 'tweet_id': tweetid, 'event': line['event'], 
                                           'support': line['support'], 'evidence_form': line['evidentiality'], 
                                           'certainty': line['certainty']}
    else: # replying tweet
        tweet2stance[threadid][tweetid] = {'thread_id': threadid, 'tweet_id': tweetid, 'event': line['event'], 
                                           'stance_to_source': line['responsetype-vs-source']}
        if 'responsetype-vs-previous' in line.keys():
            tweet2stance[threadid][tweetid]['stance_to_prev'] = line['responsetype-vs-previous']
        if 'evidentiality' in line.keys(): 
            tweet2stance[threadid][tweetid]['evidence_form'] = line['evidentiality']
        if 'certainty' in line.keys():
            tweet2stance[threadid][tweetid]['certainty'] = line['certainty']

NameError: name 'root_path' is not defined

In [ ]:
with open('./tweet_annotations.json', 'wb') as outfile:
    json.dump(tweet2stance, outfile)
#     pickle.dump(tweet2stance, handle, protocol=pickle.HIGHEST_PROTOCOL)

#### Create thread_by_event - branches from threads-structure and tweet text, grouped by events 

In [ ]:
from copy import deepcopy
# taken from BranchLSTM  https://github.com/kochkinaelena/branchLSTM
def tree2branches(root):
    node = root
    if len(list(node.values())[0])==0:
        return []
    parent_tracker = []
    parent_tracker.append(root)
    branch = []
    branches = []
    i = 0
    
    while True:
        node_name = list(node.keys())[i]
        #print node_name
        branch.append(node_name)
        # get children of the node
        first_child = list(node.values())[i]
        # actually all chldren, all tree left under this node
        if first_child != []:  # if node has children
            node = first_child      # walk down
            parent_tracker.append(node)
            siblings = list(first_child.keys())
            i = 0  # index of a current node
        else:
            branches.append(deepcopy(branch))
            i = siblings.index(node_name)  # index of a current node
            # if the node doesn't have next siblings
            while i+1 >= len(siblings):
                if node is parent_tracker[0]:  # if it is a root node
                    return branches
                del parent_tracker[-1]
                del branch[-1]
                node = parent_tracker[-1]      # walk up ... one step
                node_name = branch[-1]
                siblings = list(node.keys())
                i = siblings.index(node_name)
            i = i+1    # ... walk right
            del branch[-1]
# branches = tree2branches(tree)
# for b in branches:
#     print(b)

In [ ]:
# get general thread first from PHEME
'Note: thread2branches_by_event - having all branches as one instance'
# threads_by_event = {'event_name': [{'thread_id': threadid, 'branches': branch_arr}]} How it should look like
all_event_threads = {}

no_branch_cnt = 0
yes_branch_cnt = 0
events = [d for d in os.listdir(base_path) if d!='.DS_Store']
for event in tqdm(events):
    threads = []
    rumour_threads = [d for d in os.listdir('{}/{}/rumours'.format(base_path, event)) if d != '.DS_Store']
    non_rumour_threads = [d for d in os.listdir('{}/{}/non-rumours'.format(base_path, event)) if d != '.DS_Store']

    for r_id in rumour_threads:
        with open('{}/{}/rumours/{}/structure.json'.format(base_path, event, r_id)) as json_file:
            thread_tree = json.load(json_file)
            branches = tree2branches(thread_tree)
            if len(branches)==0:
                branches = [[r_id]]
            threads += {'thread_id': r_id, 'branches': branches},

    for r_id in non_rumour_threads:
        with open('{}/{}/non-rumours/{}/structure.json'.format(base_path, event, r_id)) as json_file:
            thread_tree = json.load(json_file)
            branches = tree2branches(thread_tree)
            if len(branches)==0:
                branches = [[r_id]]
            threads += {'thread_id': r_id, 'branches': branches},
    
    event_name = event.split('-')[0]
    all_event_threads[event_name] = threads
    print(event_name, len(threads))
    
# print("yes, no", yes_branch_cnt, no_branch_cnt)
# with open('./thread2branches_by_event.pickle', 'wb') as handle:
#     pickle.dump(all_event_threads, handle, protocol=pickle.HIGHEST_PROTOCOL)

# checking for the count
all_cnt = 0
for k in all_event_threads:
    all_cnt += len(all_event_threads[k])
    print(k, len(all_event_threads[k]))
print(all_cnt)

In [ ]:
# get general thread first from PHEME
'Note: thread2branches_by_event_split_branch - split the branches into multiple instance'
'Will end up with instance with same threadid but different branch'
# threads_by_event = {'event_name': [{'thread_id': threadid, 'branches': branch_arr}]} How it should look like
all_event_threads = {}

no_branch_cnt = 0
yes_branch_cnt = 0
events = [d for d in os.listdir(base_path) if d!='.DS_Store']
for event in tqdm(events):
    threads = []
    rumour_threads = [d for d in os.listdir('{}/{}/rumours'.format(base_path, event)) if d != '.DS_Store']
    non_rumour_threads = [d for d in os.listdir('{}/{}/non-rumours'.format(base_path, event)) if d != '.DS_Store']

    for r_id in rumour_threads:
        with open('{}/{}/rumours/{}/structure.json'.format(base_path, event, r_id)) as json_file:
            thread_tree = json.load(json_file)
            branches = tree2branches(thread_tree)
            
            if len(branches)==0:
                branches = [[r_id]]
                
            for branch in branches:
                threads += {'thread_id': r_id, 'branch': branch},

    for r_id in non_rumour_threads:
        with open('{}/{}/non-rumours/{}/structure.json'.format(base_path, event, r_id)) as json_file:
            thread_tree = json.load(json_file)
            branches = tree2branches(thread_tree)
            
            if len(branches)==0:
                branches = [[r_id]]

            for branch in branches:
                threads += {'thread_id': r_id, 'branch': branch},

    event_name = event.split('-')[0]
    all_event_threads[event_name] = threads
    print(event_name, len(threads))
    
with open('./thread2branches_by_event_split_branch.pickle', 'wb') as handle:
    pickle.dump(all_event_threads, handle, protocol=pickle.HIGHEST_PROTOCOL)

# checking for the count
all_cnt = 0
for k in all_event_threads:
    all_cnt += len(all_event_threads[k])
    print(k, len(all_event_threads[k]))
print(all_cnt)

#### ID to tweet text

In [ ]:
def process_id2text(threads, lvl_1_path, id2text):
    for r_id in threads:
        # for each rumour_thead, there is ONE source-tweet, and MULTIPLE reations

        # {base}/{event}/{rumours or non-rumour)}/{thread-id}/source-tweets/{rumour-id}.json
        with open('{}/{}/{}/{}/source-tweets/{}.json'.format(base_path, event, lvl_1_path, r_id, r_id)) as json_file:
            tweet_json = json.load(json_file)
            id2text[tweet_json['id_str'].decode("utf-8")] = tweet_json['text']
            
        reaction_tweets = [d for d in os.listdir('{}/{}/{}/{}/reactions'.format(base_path, event, lvl_1_path, r_id))
                          if 'json' in d]
        for r_tweet_id in reaction_tweets:
            with open('{}/{}/{}/{}/reactions/{}'.format(base_path, event, lvl_1_path, r_id, r_tweet_id)) as json_file:
                tweet_json = json.load(json_file)
                id2text[tweet_json['id_str'].decode("utf-8")] = tweet_json['text']
    return id2text

In [ ]:
events = [d for d in os.listdir(base_path) if d!='.DS_Store']
id2text = {}

for event in tqdm(events):
    threads = [d for d in os.listdir('{}/{}'.format(base_path, event))]

    rumour_threads = [d for d in os.listdir('{}/{}/rumours'.format(base_path, event)) if d != '.DS_Store']
    non_rumour_threads = [d for d in os.listdir('{}/{}/non-rumours'.format(base_path, event)) if d != '.DS_Store']

    id2text = process_id2text(rumour_threads, 'rumours', id2text)
    id2text = process_id2text(non_rumour_threads, 'non-rumours', id2text)

# with open('./id2text.pickle', 'wb') as handle:
#     pickle.dump(id2text, handle, protocol=pickle.HIGHEST_PROTOCOL)

#### ID to tweet info mapping 

In [ ]:
useful_tweet_fields = ['text','id_str','favorite_count','retweeted','entities','retweet_count','favorited',
                 'coordinates','created_at','place']
# entities = entities automatically detected and extracted by twitter api - provides span and text if any detected.

useful_user_fields = ['id_str', 'verified','followers_count', 'listed_count', 'statuses_count', 'description', 'friends_count', 'location',
'name', 'lang', 'favourites_count', 'screen_name', 'url', 'created_at','time_zone']
# listed_count = number of public lists that this user is a member of
# statuses_count = number of tweets/retweets
# description = self description
# favourites_count = number of Tweets this user has liked in the account’s lifetime
# url = user provided url associated with their profile

def format_tweet(tweet_obj):
    filtered_tweet_obj = {}
    for field in tweet_obj:
        field = field.decode("utf-8")

        if field == 'user':
            user_obj = {}
            for f in tweet_obj['user']:
                if f in useful_user_fields:
                    user_obj[f] = tweet_obj['user'][f]
            filtered_tweet_obj['user']=user_obj
        elif field in useful_tweet_fields:
            filtered_tweet_obj[field]=tweet_obj[field]
    
    return filtered_tweet_obj

In [ ]:
def process_id2tweet(threads, lvl_1_path, id2tweet):
    for r_id in threads:
        # for each rumour_thead, there is ONE source-tweet, and MULTIPLE reations

        # {base}/{event}/{rumours or non-rumour)}/{thread-id}/source-tweets/{rumour-id}.json
        with open('{}/{}/{}/{}/source-tweets/{}.json'.format(base_path, event, lvl_1_path, r_id, r_id)) as json_file:
            tweet_json = json.load(json_file)
            tweet_json = format_tweet(tweet_json)
            id2tweet[tweet_json['id_str'].decode("utf-8")] = tweet_json
            
        reaction_tweets = [d for d in os.listdir('{}/{}/{}/{}/reactions'.format(base_path, event, lvl_1_path, r_id))
                          if 'json' in d]
        for r_tweet_id in reaction_tweets:
            with open('{}/{}/{}/{}/reactions/{}'.format(base_path, event, lvl_1_path, r_id, r_tweet_id)) as json_file:
                tweet_json = json.load(json_file)
                tweet_json = format_tweet(tweet_json)
                id2tweet[tweet_json['id_str'].decode("utf-8")] = tweet_json
    return id2tweet

In [ ]:
events = [d for d in os.listdir(base_path) if d!='.DS_Store']
id2tweet = {}

for event in tqdm(events):
    threads = [d for d in os.listdir('{}/{}'.format(base_path, event))]
    print(event)

    rumour_threads = [d for d in os.listdir('{}/{}/rumours'.format(base_path, event)) if d != '.DS_Store']
    non_rumour_threads = [d for d in os.listdir('{}/{}/non-rumours'.format(base_path, event)) if d != '.DS_Store']

    id2tweet = process_id2tweet(rumour_threads, 'rumours', id2tweet)
    id2tweet = process_id2tweet(non_rumour_threads, 'non-rumours', id2tweet)

In [ ]:
with open('./id2tweet.pickle', 'wb') as handle:
    pickle.dump(id2tweet, handle, protocol=pickle.HIGHEST_PROTOCOL)